Review and comments -> Split to sentences

Sentences -> Split to word

Word -> Processed and get sentiment

For each sentence add up the sentiment of the words

For each document add up the sentiment of the sentences

In [39]:
import json_lines
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag
from nltk.corpus import stopwords
import string
import pandas as pd
import json
import mtranslate

In [40]:
def penn_to_wn(tag):
    """
    Convert between the PennTreebank tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

In [41]:
df3 = pd.DataFrame(columns=['Content', 'Sentiment', 'Source'])
lemmatizer = WordNetLemmatizer()
#For some reason, please returns positive sentiment because it assumes 'pleasing' but normally people just asking politely
weird_sentiment_words = ['la', 'please']

file = json.load(open('muayfit6months.json', encoding='utf-8', errors='ignore'))
for line in file['data']:
    
    try:
        #Check if key exists
        line['comments']
        #print(line['message'])
        
        for comments in line['comments']['data']:
            print("New comment")
            print('---------------------------------------')
            sentence = comments['message']
            
            #Translate to English
            sentence = mtranslate.translate(sentence, 'en', 'auto')
            
            #Split a single comment into multiple sentences
            sentences = sent_tokenize(sentence)
            
            #print(sentences)
            
            if len(sentences) == 0:
                continue
                
            overall_sentiment = 0
            
            #Sentiment for each sentence
            for s in sentences:
                    
                s = s.lower()
                s = s.replace('muayfit kota kemuning', ' ')
                s = s.replace('muayfit', ' ')
                s = re.sub(r"\byin\b", "", s)
                s = re.sub(r"\byang\b", "", s)
                
                print("Sentence: ", s)
                sentiment = 0.0
                print(s)
                
                tokenized = word_tokenize(s)
                
                new_tokenized = [x for x in tokenized if x not in weird_sentiment_words]
                tokenized = new_tokenized
                #print(tokenized)
                    
                #POS tag each word
                tagged = pos_tag(tokenized)
                print(tagged)
                
                previous_word = ""
                
                #Get the sentiment for each word
                for w, t in tagged:
                    
                    sentiment_added = False
                    
                    #Convert Penn Treebank POS tag to Wordnet POS tag
                    wn_tag = penn_to_wn(t)
                    
                    #If successfully converted, lemmatize using the Wordnet POS tag
                    #If not successfully converted, lemmatize without any POS tags
                    if wn_tag is None:
                        lemma = lemmatizer.lemmatize(w)
                    else:
                        lemma = lemmatizer.lemmatize(w, pos=wn_tag)
                        
                    #If can't lemmatize, set the word to be passed into SentiWordnet as the initial word, w
                    #If can lemmatize, set the word to be passed into Sentiwordnet as the lemmatized word, lemma
                    if not lemma:
                        print("Can't lemmatize: ", w)
                        final_word = w
                    else:
                        print("Lemmatized word: ", lemma)
                        final_word = lemma
                        
                    #Make cool and wow positive words
                    if (final_word in ['cool', 'wow', 'glad']) and (previous_word == 'not'):
                        print('Not is the previous word')
                        sentiment = sentiment - 0.5
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word in ['cool', 'wow', 'glad']):
                        print(final_word, " has a score of 0.5")
                        sentiment = sentiment + 0.5
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word == 'nice') and (previous_word == 'not'):
                        print('Not is the previous word')
                        sentiment = sentiment - 0.875
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word == 'nice'):
                        print(final_word, " has a score of 0.875")
                        sentiment = sentiment + 0.875
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word in ['terminate', 'stop', 'halt', 'cancel']) and (previous_word == 'not'):
                        sentiment = sentiment + 0.75
                        print("The word terminate, stop or halt is detected and the sentiment is -0.75")
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word in ['terminate', 'stop', 'halt', 'cancel']):
                        sentiment = sentiment - 0.75
                        print("The word terminate, stop or halt is detected and the sentiment is -0.75")
                        previous_word = final_word
                        sentiment_added = True
                        
                    #If the Wordnet POS tag exists, use it to get the synset of the word
                    if wn_tag is None:
                        synsets = wn.synsets(final_word)
                    else:
                        synsets = wn.synsets(final_word, pos=wn_tag)
                        
                    #If no synsets found, skip the word
                    if not synsets:
                        print("Synset not found")
                        continue
                        
                    # Take the first sense of the word which is the most common
                    synset = synsets[0]
                    swn_synset = swn.senti_synset(synset.name())
                    
                    if sentiment_added == False:
                        if final_word != 'fitness' and previous_word == 'not':      
                            print(final_word, " | ", synset, " | ", swn_synset, " | ", synset.definition())
                            sentiment = sentiment - swn_synset.pos_score() - swn_synset.neg_score()
                        elif final_word != 'fitness':
                            print(final_word, " | ", synset, " | ", swn_synset, " | ", synset.definition())
                            sentiment = sentiment + swn_synset.pos_score() - swn_synset.neg_score()
                        else:
                            print("The word fitness is not considered as a sentiment")
                    
                    previous_word = final_word
                
                overall_sentiment = overall_sentiment + sentiment

            df2 = pd.DataFrame([[sentence, overall_sentiment, 'Facebook Comment MuayFit']],columns=['Content', 'Sentiment', 'Source'])
                    
            df3 = pd.concat([df2,df3], ignore_index=True)
                    
            # sum greater than 0 => positive sentiment
            if overall_sentiment >= 0:
                print("This document has a positive sentiment: " + str(overall_sentiment))
            elif overall_sentiment < 0:
                print("This document has a negative sentiment: " + str(overall_sentiment))
        print()
    except:
        continue





New comment
---------------------------------------
Sentence:  ???????
???????
[('?', '.'), ('?', '.'), ('?', '.'), ('?', '.'), ('?', '.'), ('?', '.'), ('?', '.')]
Lemmatized word:  ?
Synset not found
Lemmatized word:  ?
Synset not found
Lemmatized word:  ?
Synset not found
Lemmatized word:  ?
Synset not found
Lemmatized word:  ?
Synset not found
Lemmatized word:  ?
Synset not found
Lemmatized word:  ?
Synset not found
Sentence:  ?
?
[('?', '.')]
Lemmatized word:  ?
Synset not found
This document has a positive sentiment: 0.0
New comment
---------------------------------------
Sentence:  effendey kalai ???
effendey kalai ???
[('effendey', 'NN'), ('kalai', 'NN'), ('?', '.'), ('?', '.'), ('?', '.')]
Lemmatized word:  effendey
Synset not found
Lemmatized word:  kalai
Synset not found
Lemmatized word:  ?
Synset not found
Lemmatized word:  ?
Synset not found
Lemmatized word:  ?
Synset not found
Sentence:  ?
?
[('?', '.')]
Lemmatized word:  ?
Synset not found
This document has a positive

In [46]:
df3

,Content,Sentiment,Source
0,Its a love-hate relationship,0.000,Facebook Comment Chi Fitness
1,?????????????????????,0.000,Facebook Comment Chi Fitness
2,Eid Mubarak to u too ??,-0.125,Facebook Comment Chi Fitness
3,Wow,0.500,Facebook Comment Chi Fitness
4,Arh... i miss the training. ??,-0.125,Facebook Comment Chi Fitness
5,"free for ""No.1""?",0.375,Facebook Comment Chi Fitness
6,Us,0.000,Facebook Comment Chi Fitness
7,best picture ever,0.750,Facebook Comment Chi Fitness
8,"Ethan C, you have 6 lots of solid lorrr ... ??",1.125,Facebook Comment Chi Fitness
9,"I dream of 6 packs, i wake up with 8 packs. Ho...",-0.125,Facebook Comment Chi Fitness


In [47]:
df3[df3['Sentiment'] < 0]['Content'].tolist()

['Eid Mubarak to u too ??',
 'Arh... i miss the training. ??',
 'I dream of 6 packs, i wake up with 8 packs. How Ethan C? ?????? nice packs btw ????????????',
 'Lawrence Ng OH OIIII ...',
 'She just came up from nowhere!!!! Haha!!! That speed!',
 'I like MuayThai but other than hitting my BF once in awhile I literally got no experience in punching and kicking. How do I get started? Do you have group class for beginners?',
 'terrible',
 'Shame it for me. ??????']

In [48]:
df3[df3['Sentiment'] > 0]['Content'].tolist()

['Wow',
 'free for "No.1"?',
 'best picture ever',
 'Ethan C, you have 6 lots of solid lorrr ... ??',
 "Yes. Her daughter don't like to 'bao bao'",
 'nice',
 'Look like father and son',
 "I'm glad that he is one of your small team member. Wanted to thank you for training him up. He is very disciplined and fit in any other kinds of sports!",
 'Will Wong i do this to u can ya handsome ? Ur birthday kannnn ??????',
 'Nice...',
 'Your look so fabulous',
 'If u fall...u will burst out from the window bro...lol... Ethan C',
 'Ethan C you are my idol...lol',
 'Missed the chance to make it terrific morning with u guys.',
 'Do you want to build a snowman ??',
 'Robi is looking cute',
 'Thank you for tidying them up! Can easily recognized mine. Impressive!',
 'Jin love... thank you!',
 "Pm....I'm interested.",
 'Angeline Teh wanna try out? Heard its good',
 'Pm..i interested fr me + 1 kids..',
 'Do u have trial period? If i want to join...can i join only for 3 month?',
 'Yoga to start the day! I

In [49]:
df3.to_json('muayfit_sentiment.json', orient='records')